# Introduction to data munging with Jupyter and pandas
## Open Source Bridge 2015

In [85]:
from __future__ import division

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn
import rpy2

from IPython.display import (display,
                             Image,
                             YouTubeVideo)

%matplotlib inline

## The case for open source data tools

- [Reproducibility and Transparency](http://www.nature.com/news/interactive-notebooks-sharing-the-code-1.16261)

- Cost -- compare capabilities between software you already use and open source [here](https://en.wikipedia.org/wiki/Comparison_of_statistical_packages)

- Learn more about open science at [Amy Boyle](https://twitter.com/amylouboyle)'s talk [Thursday afternoon](http://opensourcebridge.org/sessions/1535)!

## IPython/Jupyter Notebook: some cool tips for beginners

### 1a. holy moly tab completion

### 1b. built-in documentation

In [5]:
?pd.read_csv

In [ ]:
%quickref

### 2. markup!

# Markdown and HTML
<img src='ipynb.gif' align='left'></span>
<br>
<br>
<br>

# LaTeX
<span style="font-size: 24px"> $\bar{x} = \frac{\sum_{i=1}^{n}w_i\cdot x_i}{\sum_{i=1}^{n}w_i}$</span>


# Syntax highlighting for other languages
```R
x <- c(0:10, 50)
xm <- mean(x)
c(xm, mean(x, trim = 0.10))
```

### 3. [magic methods](https://ipython.org/ipython-doc/dev/interactive/magics.html)

In [4]:
%load_ext rmagic

The rmagic extension is already loaded. To reload it, use:
  %reload_ext rmagic


In [6]:
%R x <- c(0:10, 50)
%R xm <- mean(x)
%R c(xm, mean(x, trim = 0.10))

array([ 8.75,  5.5 ])

### 4. [multimedia](https://ipython.org/ipython-doc/dev/api/generated/IPython.display.html)

In [32]:
YouTubeVideo("L4Hbv4ugUWk")

### 5. Notebooks are now viewable directly [on github](https://github.com/blog/1995-github-jupyter-notebooks-3), and also exportable as PDF or HTML.

# pandas

# Input
Source: [IRS.gov](http://www.irs.gov/uac/SOI-Tax-Stats-Individual-Income-Tax-Statistics-ZIP-Code-Data-%28SOI%29)

In [52]:
?pd.read_csv

In [58]:
# read in a CSV
# specify that zipcode should be treated as a string rather than an int!
AGI = pd.read_csv('12zpallagi.csv',dtype={'zipcode': str})

In [61]:
AGI.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 166904 entries, 0 to 166903
Data columns (total 77 columns):
STATEFIPS    166904 non-null int64
STATE        166904 non-null object
zipcode      166904 non-null object
AGI_STUB     166904 non-null int64
N1           166904 non-null float64
MARS1        166904 non-null float64
MARS2        166904 non-null float64
MARS4        166904 non-null float64
PREP         166904 non-null float64
N2           166904 non-null float64
NUMDEP       166904 non-null float64
A00100       166904 non-null float64
N00200       166904 non-null float64
A00200       166904 non-null float64
N00300       166904 non-null float64
A00300       166904 non-null float64
N00600       166904 non-null float64
A00600       166904 non-null float64
N00650       166904 non-null float64
A00650       166904 non-null float64
N00900       166904 non-null float64
A00900       166904 non-null float64
SCHF         166904 non-null float64
N01000       166904 non-null float64
A01000 

# Transformation

In [62]:
# you can select columns by label or position!
AGI_column_subset = AGI[['STATE','AGI_STUB','zipcode','N1','A00100']]

In [91]:
AGI_column_subset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 166904 entries, 0 to 166903
Data columns (total 5 columns):
STATE         166904 non-null object
AGI_STUB      166904 non-null int64
zipcode       166904 non-null object
population    166904 non-null float64
amount        166904 non-null float64
dtypes: float64(2), int64(1), object(2)
memory usage: 7.6+ MB


In [66]:
# note this is inclusive!
AGI_row_subset = AGI.ix[2:4]

In [67]:
AGI_row_subset

,STATEFIPS,STATE,zipcode,AGI_STUB,N1,MARS1,MARS2,MARS4,PREP,N2,...,N09600,A09600,N06500,A06500,N10300,A10300,N11901,A11901,N11902,A11902
2,1,AL,00000,3,254280,68160,148960,31480,157230,591880,...,100,73,242260,1214154,245280,1286668,55290,113674,193020,550512
3,1,AL,00000,4,160160,23020,126140,9030,97620,420830,...,260,318,158560,1281451,158980,1342424,38890,104526,116570,388708
4,1,AL,00000,5,183320,15880,161260,4890,110520,517060,...,2350,4924,182620,3193351,182800,3319526,66910,323614,108270,430314


In [68]:
AGI_column_subset.rename(columns={'N1':'population','A00100':'amount'},inplace=True)

In [71]:
# also look into pandas.Series.unique
AGI_subset_geo = AGI[['zipcode','STATE']].drop_duplicates()

In [70]:
# group by zipcode and sum other values, resetting index
AGI_grouped = AGI_column_subset.groupby('zipcode').sum().reset_index()

In [72]:
AGI_grouped.head()

,zipcode,AGI_STUB,population,amount
0,00000,1071,142098490,9123982917
1,01001,21,8780,458716
2,01002,21,9460,732849
3,01005,21,2230,122744
4,01007,21,7320,467891


In [86]:
Image(url="http://ecdn.funzypics.com/grumpycatmemes/pics/16/OH-No--The-Cute-Kitty-Cats-Are-Attacking-Like-Aliens----The-Hunt-Is-On-Wow-Quest-Guide-.jpg")

### Delete or render null? You decide!

In [75]:
AGI_grouped['amount'].mean()

658296.02258297254

In [74]:
null_zips = AGI_grouped['zipcode'] == '00000'
AGI_grouped.loc[null_zips, 'zipcode'] = np.nan

In [76]:
AGI_grouped.head()

,zipcode,AGI_STUB,population,amount
0,NaN,1071,142098490,9123982917
1,01001,21,8780,458716
2,01002,21,9460,732849
3,01005,21,2230,122744
4,01007,21,7320,467891


In [79]:
AGI_notnull = AGI_grouped.dropna()

In [80]:
AGI_notnull['amount'].mean()

329159.88415888022

In [26]:
# make a new column 
AGI_grouped['actual_amount'] = AGI_grouped['amount'] * 1000

In [27]:
AGI_grouped['weighted_mean_AGI'] = AGI_grouped['actual_amount']/AGI_grouped['population']

In [28]:
#use anonymous functions to change every value in a column!
AGI_grouped['weighted_mean_AGI']= AGI_grouped['weighted_mean_AGI'].apply(lambda x: round(x, 0))

In [29]:
# drop columns you won't need
AGI_grouped.drop(['AGI_STUB','amount','actual_amount',],axis=1,inplace=True)

In [30]:
#merge rather than join if you want to use a common column other than the index
AGI_final = pd.merge(AGI_grouped, AGI_subset_geo, how='left', on='zipcode')

In [31]:
AGI_final.head()

,zipcode,population,weighted_mean_AGI,STATE
0,01001,8780,52246,MA
1,01002,9460,77468,MA
2,01005,2230,55042,MA
3,01007,7320,63920,MA
4,01008,640,60136,MA


In [56]:
AGI_final.sort('weighted_mean_AGI',ascending=False).head()

,zipcode,population,weighted_mean_AGI,STATE
9031,33109,250,2694776,FL
26104,94027,3220,1464534,CA
4926,19035,2040,1052019,PA
2194,10005,5580,983554,NY
9168,33480,5580,966673,FL


In [57]:
AGI_final.sort('weighted_mean_AGI').head()

,zipcode,population,weighted_mean_AGI,STATE
13238,47406,150,9860,IN
19064,64147,200,12320,MO
24549,84112,300,16823,UT
10537,38126,2280,17240,TN
12491,45225,3020,17705,OH


# References



<style>
div.text_cell_render {
font-family: "Times New Roman", serif;
...
}
</style>

[Python for Data Analysis](http://shop.oreilly.com/product/0636920023784.do) written by Wes McKinney, creator of pandas

The inimitable Julia Evans' [pandas cookbook](https://github.com/jvns/pandas-cookbook)

[Cyrille Rossant](http://ipython-books.github.io/)